# Data Handler Demo

WRDS-backed S&P500 data. Run ingestion first (requires WRDS creds):

```bash
python -m src.data_pipeline.ingestion.wrds_ingestion --root . --start 2000-01-01 --end 2025-01-01 --save-raw
```

Then execute the cells below.


## Optional: Test WRDS connectivity

If you have WRDS credentials set in `config/wrds_credentials.yml` (gitignored) or `~/.pgpass`, run the next cell to verify login.

In [6]:
import sys
from pathlib import Path

# Resolve project root whether you're running from the repo root or notebooks/
cwd = Path.cwd().resolve()
if (cwd / 'config').exists():
    PROJECT_ROOT = cwd
elif cwd.name == 'notebooks' and (cwd.parent / 'config').exists():
    PROJECT_ROOT = cwd.parent
else:
    PROJECT_ROOT = cwd.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

try:
    import wrds
except ImportError as exc:
    raise SystemExit('wrds package not installed in this env. Install via conda/pip and retry.') from exc

creds_path = PROJECT_ROOT / 'config' / 'wrds_credentials.yml'
username = password = None
if creds_path.exists():
    import yaml
    data = yaml.safe_load(creds_path.read_text()) or {}
    username, password = data.get('username'), data.get('password')

print('Connecting to WRDS...')
db = wrds.Connection(wrds_username=username, wrds_password=password)
print('Connected. Available libraries (first 10):')
print(db.list_libraries()[:10])
db.close()


Connecting to WRDS...
Loading library list...
Done
Connected. Available libraries (first 10):
['aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', 'audit_common', 'audit_corp_legal', 'auditsmp', 'auditsmp_all', 'bank', 'bank_all']


## Inspect FF factors table (wrds ff_all.factors_daily)
Check available columns and a few rows to debug missing factor fields.

In [7]:
# Requires WRDS creds; reuses PROJECT_ROOT resolution from above
import pandas as pd

try:
    import wrds
except ImportError as exc:
    raise SystemExit('wrds package not installed in this env. Install and retry.') from exc

creds_path = PROJECT_ROOT / 'config' / 'wrds_credentials.yml'
username = password = None
if creds_path.exists():
    import yaml
    data = yaml.safe_load(creds_path.read_text()) or {}
    username, password = data.get('username'), data.get('password')

db = wrds.Connection(wrds_username=username, wrds_password=password)
try:
    info = db.raw_sql('select * from ff_all.fivefactors_daily limit 5')
    display(info.head())
    print('Columns:', list(info.columns))
finally:
    db.close()


Loading library list...
Done


,date,mktrf,smb,hml,rmw,cma,rf,umd
0,1963-07-01,-0.0067,0.0,-0.0034,-0.0001,0.0016,0.0001,-0.0023
1,1963-07-02,0.0079,-0.0026,0.0026,-0.0007,-0.002,0.0001,0.0044
2,1963-07-03,0.0063,-0.0017,-0.0009,0.0018,-0.0034,0.0001,0.0038
3,1963-07-05,0.004,0.0008,-0.0027,0.0009,-0.0034,0.0001,0.0006
4,1963-07-08,-0.0063,0.0004,-0.0018,-0.0029,0.0014,0.0001,-0.0045


Columns: ['date', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd']


In [8]:
from pathlib import Path
import sys

# Resolve project root whether launched from repo root or notebooks/
cwd = Path.cwd().resolve()
if (cwd / 'config').exists():
    PROJECT_ROOT = cwd
elif cwd.name == 'notebooks' and (cwd.parent / 'config').exists():
    PROJECT_ROOT = cwd.parent
else:
    PROJECT_ROOT = cwd.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.data_pipeline.storage import LocalParquetDataHandler

handler = LocalParquetDataHandler(data_root=PROJECT_ROOT)
handler


## Inspect data source metadata
Check `data_meta/data_sources.yml` to confirm all tables came from WRDS.


In [9]:
import yaml
from pprint import pprint

log_path = PROJECT_ROOT / 'data_meta' / 'data_sources.yml'
log = yaml.safe_load(log_path.read_text()) if log_path.exists() else None
pprint(log)


{'datasets': {'analyst_consensus': {'path': 'data_processed/analyst_consensus.parquet',
                                    'source': 'wrds_tr_ibes_recdsum'},
              'analyst_ratings_history': {'path': 'data_processed/analyst_ratings_history.parquet',
                                          'source': 'wrds_det_rec'},
              'assets_master': {'path': 'data_meta/assets_master.parquet',
                                'source': 'wrds_crsp_dsenames'},
              'benchmarks': {'path': 'data_processed/benchmarks.parquet',
                             'source': 'wrds_crsp_dsp500'},
              'dividends_monthly': {'path': 'data_processed/dividends_monthly.parquet',
                                    'source': 'wrds_crsp_msedist'},
              'fundamentals_quarterly': {'path': 'data_processed/fundamentals_quarterly.parquet',
                                         'source': 'wrds_comp_funda'},
              'macro_timeseries': {'path': 'data_processed/macro_timeseri

## Dataset overview
Quick view of sources, locations, and columns for each processed dataset.

In [11]:
import yaml
from pathlib import Path
import pandas as pd
try:
    from pyarrow import parquet as pq
    def _cols(path: Path):
        return pq.read_schema(path).names
except Exception:
    def _cols(path: Path):
        return list(pd.read_parquet(path, nrows=0).columns)

log_path = PROJECT_ROOT / 'data_meta' / 'data_sources.yml'
log = yaml.safe_load(log_path.read_text()) if log_path.exists() else {}
rows = []
for name, info in (log.get('datasets') or {}).items():
    if name == 'raw':
        continue
    path = Path(info.get('path', ''))
    cols = None
    if path.exists():
        try:
            cols = _cols(path)
        except Exception as exc:
            cols = [f'<failed to read cols: {exc}>']
    rows.append({
        'dataset': name,
        'source': info.get('source'),
        'path': str(path),
        'columns': cols,
    })
pd.DataFrame(rows)


,dataset,source,path,columns
0,analyst_consensus,wrds_tr_ibes_recdsum,data_processed/analyst_consensus.parquet,None
1,analyst_ratings_history,wrds_det_rec,data_processed/analyst_ratings_history.parquet,None
2,assets_master,wrds_crsp_dsenames,data_meta/assets_master.parquet,None
3,benchmarks,wrds_crsp_dsp500,data_processed/benchmarks.parquet,None
4,dividends_monthly,wrds_crsp_msedist,data_processed/dividends_monthly.parquet,None
5,fundamentals_quarterly,wrds_comp_funda,data_processed/fundamentals_quarterly.parquet,None
6,macro_timeseries,fred_api,data_processed/macro_timeseries.parquet,None
7,prices_daily,wrds_crsp_dsf,data_processed/prices_daily.parquet,None
8,returns_daily,wrds_crsp_dsf_ret,data_processed/returns_daily.parquet,None
9,returns_monthly,wrds_crsp_msf_ret_dlret,data_processed/returns_monthly.parquet,None


In [12]:
# Universe on a specific date
handler.get_universe("2020-01-02").head()

,date,asset_id,in_universe
0,2020-01-02,10104,True
1,2020-01-02,10107,True
2,2020-01-02,10138,True
3,2020-01-02,10145,True
4,2020-01-02,10516,True


In [15]:
# Prices subset
handler.get_prices(["TSLA"], start_date="2020-01-01", end_date="2020-02-01", fields=["close", "volume"]).head()

,date,asset_id,ticker,close,volume
0,2020-01-02,93436,TSLA,430.26001,9530112.0
1,2020-01-03,93436,TSLA,443.01001,17754937.0
2,2020-01-06,93436,TSLA,451.54001,10135350.0
3,2020-01-07,93436,TSLA,469.06,18173374.0
4,2020-01-08,93436,TSLA,492.14001,31075285.0


In [16]:
# Returns subset
handler.get_returns(["TSLA"], start_date="2020-01-02", end_date="2020-02-01").head()

,date,asset_id,ticker,ret_1d
0,2020-01-02,93436,TSLA,0.028518
1,2020-01-03,93436,TSLA,0.029633
2,2020-01-06,93436,TSLA,0.019255
3,2020-01-07,93436,TSLA,0.038801
4,2020-01-08,93436,TSLA,0.049205


In [17]:
# Fundamentals
handler.get_fundamentals(["AAPL"], start_date="2019-01-01", end_date="2021-12-31").head()

,report_date,asset_id,revenue,sales,net_income,total_assets,common_equity,total_debt_long_term,preferred_stock,cash_flow_from_operations,capital_expenditures,research_and_development
0,2019-09-30,14593.0,260174.0,260174.0,55256.0,338516.0,90488.0,91807.0,0.0,69391.0,10495.0,16217.0
1,2020-09-30,14593.0,274515.0,274515.0,57411.0,323888.0,65339.0,107049.0,0.0,80674.0,7309.0,18752.0
2,2021-09-30,14593.0,365817.0,365817.0,94680.0,351002.0,63090.0,119381.0,0.0,104038.0,11085.0,21914.0


In [18]:
# Macro series sample
handler.get_macro(start_date="2019-01-01", end_date="2021-12-31").head()

,date,series_name,value
0,2019-01-01,CPIAUCSL,252.5610
1,2019-01-01,INDPRO,103.4021
2,2019-01-01,UNRATE,4.0000
3,2019-02-01,CPIAUCSL,253.3190
4,2019-02-01,INDPRO,102.8371


## Test FRED API
Uses `config/fred_credentials.yml` or `FRED_API_KEY` to pull a small sample.

In [22]:
import os
import requests

api_key = None
cred_path = PROJECT_ROOT / 'config' / 'fred_credentials.yml'
if cred_path.exists():
    import yaml
    data = yaml.safe_load(cred_path.read_text()) or {}
    api_key = data.get('api_key')
api_key = api_key or os.environ.get('FRED_API_KEY')

params = {
    'series_id': 'CPIAUCSL',
    'observation_start': '2020-01-01',
    'observation_end': '2020-12-31',
    'file_type': 'json',
}
if api_key:
    params['api_key'] = api_key
resp = requests.get('https://api.stlouisfed.org/fred/series/observations', params=params, timeout=10)
resp.raise_for_status()
obs = resp.json().get('observations', [])[:5]
obs


[{'realtime_start': '2025-11-25',
  'realtime_end': '2025-11-25',
  'date': '2020-01-01',
  'value': '259.127'},
 {'realtime_start': '2025-11-25',
  'realtime_end': '2025-11-25',
  'date': '2020-02-01',
  'value': '259.250'},
 {'realtime_start': '2025-11-25',
  'realtime_end': '2025-11-25',
  'date': '2020-03-01',
  'value': '258.076'},
 {'realtime_start': '2025-11-25',
  'realtime_end': '2025-11-25',
  'date': '2020-04-01',
  'value': '256.032'},
 {'realtime_start': '2025-11-25',
  'realtime_end': '2025-11-25',
  'date': '2020-05-01',
  'value': '255.802'}]

In [19]:
# Style factor returns sample
handler.get_style_factor_returns(start_date="2019-01-01", end_date="2021-12-31").head()

,date,ret,factor_name
0,2019-01-02,0.000025,CMA
1,2019-01-02,0.000114,HML
2,2019-01-02,0.000022,MKT
3,2019-01-02,-0.000002,MOM
4,2019-01-02,-0.000016,RMW


In [24]:
# Benchmark returns sample
handler.get_benchmark_returns("^GSPC", start_date="2019-01-01", end_date="2021-12-31").head()

,date,benchmark_name,level,ret
0,2019-01-02,^GSPC,248.711268,0.001449
1,2019-01-03,^GSPC,242.696186,-0.024185
2,2019-01-04,^GSPC,251.047604,0.034411
3,2019-01-07,^GSPC,252.85113,0.007184
4,2019-01-08,^GSPC,255.332105,0.009812


## Inspect CRSP delist returns
Check availability of `crsp.dlret` and `crsp.msedelist` (or crspsamp fallback).

In [25]:
import pandas as pd
try:
    import wrds
except ImportError as exc:
    raise SystemExit('wrds package not installed in this env. Install and retry.') from exc

creds_path = PROJECT_ROOT / 'config' / 'wrds_credentials.yml'
username = password = None
if creds_path.exists():
    import yaml
    data = yaml.safe_load(creds_path.read_text()) or {}
    username, password = data.get('username'), data.get('password')

db = wrds.Connection(wrds_username=username, wrds_password=password)
try:
    try:
        dl = db.raw_sql('select * from crspsamp_all.stkdelists limit 5')
        print('crspsamp_all.stkdelists rows:', len(dl))
        display(dl)
    except Exception as e1:
        print('crspsamp_all.stkdelists unavailable:', e1)
    try:
        dlm = db.raw_sql('select * from crsp.msedelist limit 5')
        print('crsp.msedelist rows:', len(dlm))
        display(dlm)
    except Exception as e2:
        print('crsp.msedelist unavailable:', e2)
finally:
    db.close()


Loading library list...
Done
crspsamp_all.stkdelists rows: 5


,permno,delistingdt,deldtprc,deldtprcflg,delactiontype,delstatustype,delreasontype,delpaymenttype,delpermno,delpermco,delret,delretmisstype,delnextdt,delnextprc,delnextprcflg,delamtdt,deldivamt,deldistype,deldlydt
0,10823.0,1998-05-29,33.6875,TR,MER,FPAY,UNAV,STK,24360.0,21116.0,0.003711,NA,<NA>,<NA>,DA,1998-06-01,33.8125,D1,1998-06-01
1,12079.0,2009-06-01,0.75,TR,GDR,VCL,BKPY,PRCF,0.0,0.0,-0.186667,NA,2009-06-02,0.61,DP,2009-06-02,0.0,NO,2009-06-02
2,14218.0,1989-04-28,85.375,TR,MER,FPAY,UNAV,UNAV,0.0,0.0,0.023207,NA,<NA>,<NA>,DA,1989-05-01,87.3563,D2,1989-05-01
3,14322.0,2005-03-24,50.04,TR,MER,FPAY,UNAV,CST,89757.0,44118.0,0.194944,NA,<NA>,<NA>,DA,2005-03-28,59.795,D2,2005-03-28
4,18948.0,1996-12-06,64.0,TR,MER,FPAY,UNAV,STK,84381.0,31847.0,0.024184,NA,<NA>,<NA>,DA,1996-12-09,65.54775,D2,1996-12-09


crsp.msedelist rows: 5


,permno,dlstdt,dlstcd,nwperm,nwcomp,nextdt,dlamt,dlretx,dlprc,dlpdt,dlret,permco,compno,issuno,hexcd,hsiccd,cusip,acperm,accomp
0,10000,1987-06-11,560,0,0,1987-06-12,0.21875,0.0,-0.21875,1987-06-12,0.0,7952,60007905,10396,3,3990,68391610,<NA>,<NA>
1,10001,2017-08-03,233,0,0,<NA>,13.1,0.011583,0.0,2017-08-04,0.011583,7953,60007906,10398,2,4925,36720410,<NA>,<NA>
2,10002,2013-02-15,231,35263,1658,<NA>,3.0125,0.046007,0.0,2013-02-19,0.046007,7954,60007907,10399,3,6020,05978R10,<NA>,<NA>
3,10003,1995-12-15,231,10569,8477,<NA>,5.4488,0.01373,0.0,1995-12-18,0.01373,7957,60007910,10404,3,6020,39031810,<NA>,<NA>
4,10005,1991-07-11,560,0,0,1991-07-12,0.14063,0.125,-0.14063,1991-07-12,0.125,7961,60007914,10410,3,1310,95815510,<NA>,<NA>


## Inspect Compustat security table for IPO fields
List available columns in `comp.security` (check for IPO date equivalent).

In [20]:
import pandas as pd
try:
    import wrds
except ImportError as exc:
    raise SystemExit('wrds package not installed in this env. Install and retry.') from exc

creds_path = PROJECT_ROOT / 'config' / 'wrds_credentials.yml'
username = password = None
if creds_path.exists():
    import yaml
    data = yaml.safe_load(creds_path.read_text()) or {}
    username, password = data.get('username'), data.get('password')

db = wrds.Connection(wrds_username=username, wrds_password=password)
try:
    sample = db.raw_sql('select * from comp.security limit 5')
    print('Columns in comp.security:', list(sample.columns))
    display(sample)
except Exception as exc:
    print('comp.security unavailable:', exc)
finally:
    db.close()


Loading library list...
Done
Columns in comp.security: ['tic', 'gvkey', 'iid', 'cusip', 'dlrsni', 'dsci', 'epf', 'exchg', 'excntry', 'ibtic', 'isin', 'secstat', 'sedol', 'tpci', 'dldtei', 'curr_sp500_flag']


,tic,gvkey,iid,cusip,dlrsni,dsci,epf,exchg,excntry,ibtic,isin,secstat,sedol,tpci,dldtei,curr_sp500_flag
0,AE.2,001000,01,000032102,09,COM USD1,<NA>,12,USA,<NA>,<NA>,I,<NA>,0,1978-06-30,0.0
1,AMFD.,001001,01,000165100,01,COM USD.01,<NA>,14,USA,AMFD,US0001651001,I,<NA>,0,1986-07-31,0.0
2,AAIC.1,001002,01,000352104,01,COM USD1,<NA>,13,USA,<NA>,<NA>,I,<NA>,0,1977-03-31,0.0
3,ANTQ,001003,01,000354100,10,COM USD.10,<NA>,19,USA,ANTQ,US0003541002,I,2791283,0,2017-10-12,0.0
4,AIR,001004,01,000361105,<NA>,COM USD1,<NA>,11,USA,AIR,US0003611052,A,2001119,0,<NA>,0.0


In [21]:
# Diagnostics for I/B/E/S idxref availability
import pandas as pd
try:
    import wrds
    db = wrds.Connection(wrds_username=username, wrds_password=password)
    ibes_tables = db.list_tables('ibes')
    print(f"Found {len(ibes_tables)} ibes tables, sample: {ibes_tables[:10]}")
    if 'idxref' in ibes_tables:
        try:
            row_ct = db.get_row_count('ibes.idxref')
            print(f"ibes.idxref row count: {row_ct}")
            preview = db.raw_sql("select * from ibes.idxref limit 5")
            display(preview.head())
        except Exception as exc:
            print(f"Failed to query ibes.idxref: {exc}")
    else:
        print("ibes.idxref not present; check entitlement or alternative schemas (e.g., ibes.idxref or library-specific mirrors).")
except Exception as exc:
    print(f"WRDS diagnostic failed: {exc}")


Loading library list...
Done
Found 194 ibes tables, sample: ['act_epsint', 'act_epsus', 'act_xepsint', 'act_xepsus', 'actg', 'actgu', 'actpsum_epsint', 'actpsum_epsus', 'actpsum_xepsint', 'actpsum_xepsus']
ibes.idxref not present; check entitlement or alternative schemas (e.g., ibes.idxref or library-specific mirrors).


In [28]:
# Inspect IBES schemas/columns to identify available mapping fields
import pandas as pd
try:
    schemas = [s for s in db.list_libraries() if 'ibes' in s.lower()]
    print('IBES-like schemas:', schemas)
    for schema in schemas:
        tables = db.list_tables(schema)
        if 'idsum' in tables:
            cols = db.describe_table(schema, 'idsum')
            print(f"{schema}.idsum columns:")
            print(cols[['name','type']])
            sample = db.raw_sql(f"select * from {schema}.idsum limit 3")
            display(sample.head())
        if 'iclink' in tables:
            cols = db.describe_table(schema, 'iclink')
            print(f"{schema}.iclink columns:")
            print(cols[['name','type']])
            sample = db.raw_sql(f"select * from {schema}.iclink limit 3")
            display(sample.head())
except Exception as exc:
    print(f"WRDS schema/column inspection failed: {exc}")


IBES-like schemas: ['ibes', 'ibessamp_kpi', 'tr_ibes', 'wrdsapps_finratio_ibes', 'wrdsapps_finratio_ibes_ccm', 'wrdsapps_link_crsp_ibes']
Approximately 280583 rows in ibes.idsum.
ibes.idsum columns:
        name              type
0     ticker        VARCHAR(6)
1      cusip        VARCHAR(8)
2      oftic        VARCHAR(8)
3      cname       VARCHAR(32)
4     dilfac  DOUBLE PRECISION
5        pdi        VARCHAR(1)
6     ccopcf        VARCHAR(1)
7    tnthfac  DOUBLE PRECISION
8   instrmnt        VARCHAR(1)
9     exchcd        VARCHAR(1)
10   country        VARCHAR(2)
11  compflag        VARCHAR(1)
12    usfirm          SMALLINT
13    sdates              DATE


,ticker,cusip,oftic,cname,dilfac,pdi,ccopcf,tnthfac,instrmnt,exchcd,country,compflag,usfirm,sdates
0,0000,87482X10,TLMR,TALMER BANCORP INC,1.0,P,<NA>,0.0,S,F,NA,<NA>,1,2014-02-20
1,0000,87482X10,TLMR,TALMER BANCORP INC,1.0,D,<NA>,0.0,S,F,NA,<NA>,1,2014-03-20
2,0001,26878510,EPE,EP ENERGY CORP,1.0,P,<NA>,0.0,S,A,NA,<NA>,1,2014-02-20


Approximately 280583 rows in tr_ibes.idsum.
tr_ibes.idsum columns:
        name              type
0     ticker        VARCHAR(6)
1      cusip        VARCHAR(8)
2      oftic        VARCHAR(8)
3      cname       VARCHAR(32)
4     dilfac  DOUBLE PRECISION
5        pdi        VARCHAR(1)
6     ccopcf        VARCHAR(1)
7    tnthfac  DOUBLE PRECISION
8   instrmnt        VARCHAR(1)
9     exchcd        VARCHAR(1)
10   country        VARCHAR(2)
11  compflag        VARCHAR(1)
12    usfirm          SMALLINT
13    sdates              DATE


,ticker,cusip,oftic,cname,dilfac,pdi,ccopcf,tnthfac,instrmnt,exchcd,country,compflag,usfirm,sdates
0,0000,87482X10,TLMR,TALMER BANCORP INC,1.0,P,<NA>,0.0,S,F,NA,<NA>,1,2014-02-20
1,0000,87482X10,TLMR,TALMER BANCORP INC,1.0,D,<NA>,0.0,S,F,NA,<NA>,1,2014-03-20
2,0001,26878510,EPE,EP ENERGY CORP,1.0,P,<NA>,0.0,S,A,NA,<NA>,1,2014-02-20


In [22]:
# Inspect availability of tr_ibes_id and list IBES tables
try:
    tables = db.list_tables('ibes')
    print('Number of ibes tables:', len(tables))
    print('Sample:', tables[:20])
    print('Contains tr_ibes_id?', 'tr_ibes_id' in tables)
    if 'tr_ibes_id' in tables:
        cols = db.describe_table('ibes', 'tr_ibes_id')
        print('tr_ibes_id columns:')
        print(cols[['name','type']])
        sample = db.raw_sql('select * from ibes.tr_ibes_id limit 5')
        display(sample.head())
except Exception as exc:
    print(f"WRDS tr_ibes_id inspection failed: {exc}")


Number of ibes tables: 194
Sample: ['act_epsint', 'act_epsus', 'act_xepsint', 'act_xepsus', 'actg', 'actgu', 'actpsum_epsint', 'actpsum_epsus', 'actpsum_xepsint', 'actpsum_xepsus', 'actpsumu_epsint', 'actpsumu_epsus', 'actpsumu_xepsint', 'actpsumu_xepsus', 'actu_epsint', 'actu_epsus', 'actu_xepsint', 'actu_xepsus', 'adj', 'adjsum']
Contains tr_ibes_id? False


## DataHandler usage examples (all accessors)
Below are quick examples for each getter on `LocalParquetDataHandler`. Adjust tickers/dates as needed after running ingestion.


In [23]:
from pathlib import Path
import pandas as pd
from src.data_pipeline.storage import LocalParquetDataHandler

data_root = Path('..').resolve()  # adjust if needed
handler = LocalParquetDataHandler(data_root)

# Prices (daily)
prices = handler.get_prices(['AAPL', 'MSFT'], start_date='2022-01-03', end_date='2022-01-07')
display(prices.head())

# Returns (daily)
rets = handler.get_returns(['AAPL'], start_date='2022-01-03', end_date='2022-01-07')
display(rets.head())

# Fundamentals (quarterly)
funda = handler.get_fundamentals(['AAPL'], start_date='2021-01-01', end_date='2023-12-31')
display(funda.head())

# Analyst consensus
cons = handler.get_analyst_consensus(['AAPL'], start_date='2021-01-01', end_date='2023-12-31')
display(cons.head())

# Analyst rating history
hist = handler.get_analyst_ratings_history(['AAPL'], start_date='2021-01-01', end_date='2023-12-31')
display(hist.head())

# Macro series
macro = handler.get_macro('2022-01-01', '2022-03-31')
display(macro.head())

# Style factor returns
factors = handler.get_style_factor_returns('2022-01-01', '2022-01-10')
display(factors.head())

# Benchmark returns
bench = handler.get_benchmark_returns('^GSPC', '2022-01-01', '2022-01-10')
display(bench.head())

# Universe snapshot
univ = handler.get_universe('2022-01-03')
display(univ.head())


,date,asset_id,open,high,low,close,cfacpr,ret,shrout,volume,adj_close,ticker
0,2022-01-03,10107,335.35001,338.0,329.78,334.75,1.0,-0.004668,7507980.0,28823566.0,334.75,MSFT
1,2022-01-03,14593,177.83,182.88,177.71001,182.00999,1.0,0.025004,16344923.0,104656435.0,182.00999,AAPL
2,2022-01-04,10107,334.82999,335.20001,326.12,329.01001,1.0,-0.017147,7507980.0,32626156.0,329.01001,MSFT
3,2022-01-04,14593,182.63,182.94,179.12,179.7,1.0,-0.012692,16344923.0,99155535.0,179.7,AAPL
4,2022-01-05,10107,325.85999,326.07001,315.98001,316.38,1.0,-0.038388,7507980.0,39957778.0,316.38,MSFT


,date,asset_id,ticker,ret_1d
0,2022-01-03,14593,AAPL,0.025004
1,2022-01-04,14593,AAPL,-0.012692
2,2022-01-05,14593,AAPL,-0.0266
3,2022-01-06,14593,AAPL,-0.016693
4,2022-01-07,14593,AAPL,0.000988


,report_date,asset_id,revenue,sales,net_income,total_assets,common_equity,total_debt_long_term,preferred_stock,cash_flow_from_operations,capital_expenditures,research_and_development
0,2021-09-30,14593.0,365817.0,365817.0,94680.0,351002.0,63090.0,119381.0,0.0,104038.0,11085.0,21914.0
1,2022-09-30,14593.0,394328.0,394328.0,99803.0,352755.0,50672.0,109707.0,0.0,122151.0,10708.0,26251.0
2,2023-09-30,14593.0,383285.0,383285.0,96995.0,352583.0,62146.0,106548.0,0.0,110543.0,10959.0,29915.0


,date,asset_id,ticker,mean_rating,median_rating,stdev_rating,num_analysts,buy_percent,hold_percent,sell_percent,num_up,num_down,usfirm,ibes_official_ticker,ibes_cusip,company_name
0,2021-01-14,14593,AAPL,2.05,2.0,0.9,40.0,75.0,20.0,5.0,0.0,0.0,1,AAPL,03783310,None
1,2021-02-18,14593,AAPL,2.03,2.0,0.89,40.0,77.5,17.5,5.0,1.0,0.0,1,AAPL,03783310,None
2,2021-03-18,14593,AAPL,2.03,2.0,0.89,40.0,77.5,17.5,5.0,0.0,0.0,1,AAPL,03783310,None
3,2021-04-15,14593,AAPL,2.0,2.0,0.88,40.0,80.0,15.0,5.0,1.0,0.0,1,AAPL,03783310,None
4,2021-05-20,14593,AAPL,1.98,2.0,0.82,41.0,80.49,17.07,2.44,1.0,0.0,1,AAPL,03783310,None


,date,asset_id,analyst_id,ticker,rating,action_code,rating_text,statistic_date
0,2021-01-12,14593,182826.0,AAPL,3,3,HOLD,2021-01-12
1,2021-01-19,14593,194536.0,AAPL,2,2,BUY,2021-01-19
2,2021-01-28,14593,166271.0,AAPL,2,1,BUY,2021-01-28
3,2021-03-26,14593,71449.0,AAPL,2,2,BUY,2021-03-26
4,2021-04-29,14593,77028.0,AAPL,3,2,HOLD,2021-04-29


,date,series_name,value
0,2022-01-01,CPIAUCSL,282.5420
1,2022-01-01,INDPRO,100.1856
2,2022-01-01,UNRATE,4.0000
3,2022-02-01,CPIAUCSL,284.5250
4,2022-02-01,INDPRO,100.8064


,date,ret,factor_name
0,2022-01-03,0.000053,CMA
1,2022-01-03,0.000077,HML
2,2022-01-03,0.000072,MKT
3,2022-01-03,-0.000001,MOM
4,2022-01-03,-0.000117,RMW


,date,benchmark_name,level,ret
0,2022-01-03,^GSPC,501.479112,0.007068
1,2022-01-04,^GSPC,501.035804,-0.000884
2,2022-01-05,^GSPC,491.292161,-0.019447
3,2022-01-06,^GSPC,490.764513,-0.001074
4,2022-01-07,^GSPC,488.776917,-0.00405


,date,asset_id,in_universe
0,2022-01-03,10104,True
1,2022-01-03,10107,True
2,2022-01-03,10138,True
3,2022-01-03,10145,True
4,2022-01-03,10516,True


### Data integrity quick check
Run this cell after ingestion to summarize shapes, date ranges, missing/extra columns vs `reference/field_manifest.csv`, duplicates on natural keys, and top null rates. Paste the printed output back if you want a review.

In [24]:
import pandas as pd
from pathlib import Path

root = Path('..').resolve()  # repo root relative to notebook
manifest = pd.read_csv(root / 'reference/field_manifest.csv')

dataset_paths = manifest[['dataset', 'path']].drop_duplicates().set_index('dataset')['path'].to_dict()
dataset_paths.update({
    'assets_master': 'data_meta/assets_master.parquet',
    'universe_sp500': 'data_meta/universe_sp500.parquet',
    'trading_calendar': 'data_meta/trading_calendar.parquet',
    'risk_free': 'data_processed/risk_free.parquet',
})

key_cols = {
    'prices_daily': ['date', 'asset_id'],
    'returns_daily': ['date', 'asset_id'],
    'returns_monthly': ['date', 'asset_id'],
    'fundamentals_quarterly': ['report_date', 'asset_id'],
    'analyst_consensus': ['date', 'asset_id'],
    'analyst_ratings_history': ['date', 'asset_id', 'analyst_id'],
    'macro_timeseries': ['date', 'series_name'],
    'style_factor_returns': ['date', 'factor_name'],
    'benchmarks': ['date', 'benchmark_name'],
    'dividends_monthly': ['date', 'asset_id'],
    'universe_sp500': ['date', 'asset_id'],
    'assets_master': ['asset_id'],
    'trading_calendar': ['date'],
    'risk_free': ['date'],
}


def summarize(dataset, rel_path, expected_cols):
    path = root / rel_path
    if not path.exists():
        return {'dataset': dataset, 'path': rel_path, 'status': 'missing file'}
    df = pd.read_parquet(path)
    date_cols = [c for c in df.columns if 'date' in c]
    date_ranges = {c: (pd.to_datetime(df[c]).min(), pd.to_datetime(df[c]).max()) for c in date_cols}
    expected = expected_cols if expected_cols is not None else []
    missing_cols = sorted(set(expected) - set(df.columns))
    extra_cols = sorted(set(df.columns) - set(expected)) if expected_cols is not None else []
    key = key_cols.get(dataset)
    dupes = int(df.duplicated(key).sum()) if key and all(k in df.columns for k in key) else 'n/a'
    nulls = (df.isna().mean() * 100).round(2)
    top_nulls = nulls[nulls > 0].sort_values(ascending=False).head(5).to_dict()
    return {
        'dataset': dataset,
        'path': rel_path,
        'status': 'ok',
        'shape': df.shape,
        'n_assets': df['asset_id'].nunique() if 'asset_id' in df.columns else 'n/a',
        'n_tickers': df['ticker'].nunique() if 'ticker' in df.columns else 'n/a',
        'date_ranges': date_ranges,
        'missing_cols': missing_cols,
        'extra_cols': extra_cols,
        'duplicates_on_key': dupes,
        'top_null_pct': top_nulls,
    }


results = []
for name, rel_path in dataset_paths.items():
    expected_cols = manifest.loc[manifest['dataset'] == name, 'column'].tolist() or None
    results.append(summarize(name, rel_path, expected_cols))

for r in results:
    print(f"{r['dataset']} ({r['path']}):")
    print(f"  status: {r['status']}")
    if r['status'] != 'ok':
        continue
    print(f"  shape: {r['shape']}, assets: {r['n_assets']}, tickers: {r['n_tickers']}")
    for col, (dmin, dmax) in r['date_ranges'].items():
        print(f"  {col} range: {pd.to_datetime(dmin).date()} → {pd.to_datetime(dmax).date()}")
    print(f"  missing vs manifest: {r['missing_cols'] or 'none'}")
    print(f"  extra cols: {r['extra_cols'] or 'none'}")
    print(f"  duplicates on key: {r['duplicates_on_key']}")
    print(f"  top null %: {r['top_null_pct'] or 'none'}")


prices_daily (data_processed/prices_daily.parquet):
  status: ok
  shape: (4677236, 12), assets: 1087, tickers: 1052
  date range: 2000-01-03 → 2024-12-31
  missing vs manifest: none
  extra cols: none
  duplicates on key: 0
  top null %: {'open': 0.36, 'ret': 0.25, 'high': 0.24, 'low': 0.24, 'close': 0.24}
returns_daily (data_processed/returns_daily.parquet):
  status: ok
  shape: (4677236, 4), assets: 1087, tickers: 1052
  date range: 2000-01-03 → 2024-12-31
  missing vs manifest: none
  extra cols: none
  duplicates on key: 0
  top null %: {'ret_1d': 0.25}
returns_monthly (data_processed/returns_monthly.parquet):
  status: ok
  shape: (223670, 7), assets: 1087, tickers: n/a
  date range: 2000-01-31 → 2024-12-31
  missing vs manifest: none
  extra cols: none
  duplicates on key: 0
  top null %: {'ret': 0.65, 'ret_1m': 0.65, 'price': 0.52, 'volume': 0.35, 'shrout': 0.11}
dividends_monthly (data_processed/dividends_monthly.parquet):
  status: ok
  shape: (51202, 8), assets: 1031, ticke